In [1]:
import pickle
import matplotlib.pyplot as plt

f = open('./benchmark_data/quantum_plotter_input_6_qubits_6_clusters_sametotal.p', 'rb' )
benchmarks = []
while 1:
    try:
        benchmarks.append(pickle.load(f))
    except EOFError:
        break

In [8]:
circuit_idx = 0
repetition_idx = 0
benchmark = benchmarks[circuit_idx][repetition_idx]
num_shots,searcher_time,circ,evaluations,classical_time,quantum_time,uniter_time = benchmark



┌───┐      ┌───┐                                        »
 q2_0: |0>┤ H ├──────┤ T ├────────────────────────────────────────»
          ├───┤      ├───┤                                        »
 q2_1: |0>┤ H ├──────┤ T ├─────────────────────■──────────────────»
          ├───┤      ├───┤                     │                  »
 q2_2: |0>┤ H ├──────┤ T ├─────────────────────┼──────────────────»
          ├───┤      ├───┤                     │                  »
 q2_3: |0>┤ H ├──────┤ T ├─────────────────────┼──■───────────────»
          ├───┤      ├───┤                     │  │ ┌────────────┐»
 q2_4: |0>┤ H ├──────┤ T ├─■───────────────────┼──┼─┤ Ry(1.5708) ├»
          ├───┤      └───┘ │    ┌────────────┐ │  │ └────────────┘»
 q2_5: |0>┤ H ├─■──────────┼────┤ Rx(1.5708) ├─■──┼───────────────»
          ├───┤ │    ┌───┐ │    └────────────┘    │ ┌────────────┐»
 q2_6: |0>┤ H ├─┼────┤ T ├─┼──■───────────────────┼─┤ Ry(1.5708) ├»
          ├───┤ │    └───┘ │  │ ┌────────────┐    │ └─────